### Author: Ally Sprik
### Last-updated: 25-02-2024

Goal of this notebook is to explore and clean the Tubingen dataset



In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel('../0. Data/Tubingen/Data_ENDORISK_Validation_MolCLass_TUE.xlsx')

In [ ]:
df['Adjuvant therapy'].value_counts(dropna=False)

Create month to follow up column

In [ ]:
# create months to follow up
df['months_to_follow_up'] = (df['Date last Follow-Up'] - df['date_of_dx']).dt.days / 30.5

Change the column names of _cure to preop en _he to postop, for clarity

In [ ]:
# Change column names all _cur = Preop, all _he = Postop
for column in df.columns:
	if column.__contains__('_cur'):
		df.rename(columns={column: column.replace('_cur', '_PREOP')}, inplace=True)
	elif column.__contains__('_he'):
		df.rename(columns={column: column.replace('_he', '_POSTOP')}, inplace=True)


Correct the labels of the histochemical columns	

In [ ]:
# Replace 2 with 1 for L1CAM, PR, ER, and P53
df['L1CAM_PREOP'].replace(2, 1, inplace=True)
df['L1CAM_POSTOP'].replace(2, 1, inplace=True)
df['PR_PREOP'].replace(2, 1, inplace=True)
df['PR_POSTOP'].replace(2, 1, inplace=True)
df['ER_PREOP'].replace(2, 1, inplace=True)
df['ER_POSTOP'].replace(2, 1, inplace=True)
df['p53_PREOP'].replace(2, 1, inplace=True)
df['p53_POSTOP'].replace(2, 1, inplace=True)


Parse the date columns to datetime so they can be used

In [ ]:
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')
df['date_of_dx'] = pd.to_datetime(df['date_of_dx'], format='%Y-%m-%d')
df['Date last Follow-Up'] = pd.to_datetime(df['Date last Follow-Up'], format='%Y-%m-%d')

df['age_at_dx'] = (df['date_of_dx'] - df['dob']).dt.days / 365.25

Generate a histogram for the age at diagnosis

In [ ]:
# Plot histogram of age
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df['age_at_dx'], bins=20)

Create a table with the counts and percentages of peroperative grade 

In [ ]:
pd.DataFrame([df['Grade_PREOP'].value_counts(dropna=False), round(df['Grade_PREOP'].value_counts(normalize=True)*100, 1)])

Create a table with the counts and percentages of ER status

In [ ]:
df['ER_PREOP'].replace(2, 1, inplace=True)
pd.DataFrame([df['ER_PREOP'].value_counts(dropna=False),round(df['ER_PREOP'].value_counts(dropna=True, normalize=True) * 100,1)])

Create a table with the counts and percentages of PR status

In [ ]:
df['PR_PREOP'].replace(2, 1, inplace=True)
pd.DataFrame([df['PR_PREOP'].value_counts(dropna=False),
			  round(df['PR_PREOP'].value_counts(dropna=True, normalize=True) * 100, 1)])

Create a binary column for the preoperative CA-125 level

In [ ]:
# Replace everything below 35 with <35 and everything above 35 with >35
df['CA125_bi'] = df['Pre-operative CA-125 level '].apply(lambda x: '<=35' if x <=35 else ('>35' if x > 35  else x))
df['CA125_bi'].value_counts(dropna=False)

Create a binary column for the preoperative platelet count

In [ ]:
df['Pre-operative platelet count '] = df['Pre-operative platelet count '].apply(lambda x: '<400' if x <400 else ('>=400' if x >= 400  else x))
df['Pre-operative platelet count '].value_counts(dropna=False)

Create a risk level column for the preoperative risk level based on the preoperative grade, PR, and p53

In [ ]:
df['Risk_level_PREOP'] = np.nan
df['Risk_level_PREOP'] = df['Risk_level_PREOP'].astype(object)

for i in range(len(df)):
	if pd.isna(df['Grade_PREOP'][i]):
		df['Risk_level_PREOP'][i] = 'Unknown'
	elif df['Grade_PREOP'].values[i] == 'Grade 1':
		df['Risk_level_PREOP'][i] = 'Low'
	elif df['Grade_PREOP'].values[i] == 'Grade 3':
		df['Risk_level_PREOP'][i] = 'High'
	elif df['Grade_PREOP'].values[i] == 'Grade 2':
		if df['PR_PREOP'].values[i] == 0:
			df['Risk_level_PREOP'][i] = 'High'
		else:
			if df['p53_POSTOP'].values[i] == 1:
				df['Risk_level_PREOP'][i] = 'High'
			else:
				df['Risk_level_PREOP'][i] = 'Low'
	else:
		df['Risk_level_PREOP'][i] = 'Unknown'


Create a risk level column for the postoperative risk level based on the postoperative grade, PR, and p53

In [ ]:
df['Risk_level_POSTOP'] = np.nan
df['Risk_level_POSTOP'] = df['Risk_level_POSTOP'].astype(object)

for i in range(len(df)):
    if pd.isna(df['Grade_POSTOP'][i]):
        df['Risk_level_POSTOP'][i] = 'Unknown'
    elif df['Grade_POSTOP'].values[i] == 'Grade 1':
        df['Risk_level_POSTOP'][i] = 'Low'
    elif df['Grade_POSTOP'].values[i] == 'Grade 3':
        df['Risk_level_POSTOP'][i] = 'High'
    elif df['Grade_POSTOP'].values[i] == 'Grade 2':
        if df['PR_POSTOP'].values[i] == 0:
            df['Risk_level_POSTOP'][i] = 'High'
        else:
            if df['p53_POSTOP'].values[i] == 'Mutated':
                df['Risk_level_POSTOP'][i] = 'High'
            else:
                df['Risk_level_POSTOP'][i] = 'Low'
    else:
        df['Risk_level_PREOP'][i] = 'Unknown'



Create a risk level column for the postoperative risk level based on the only biomarkers

In [ ]:
df['Risk_level_BM'] = np.nan
df['Risk_level_BM'] = df['Risk_level_BM'].astype(object)

for i in range(len(df)):
	if df['PR_PREOP'].values[i] == 0:
		df['Risk_level_BM'][i] = 'High'
	elif df['p53_PREOP'].values[i] == 'Mutated':
		df['Risk_level_BM'][i] = 'High'
	elif df['Pre-operative CA-125 level '].values[i] == '>35':
		df['Risk_level_BM'][i] = 'High'
	elif df['p53_PREOP'].values[i] == 'Wildtype' or df['PR_PREOP'].values[i] > 0 or \
			df['Pre-operative CA-125 level '].values[i] == '<=35':
		df['Risk_level_BM'][i] = 'Low'
	else:
		df['Risk_level_BM'][i] = 'Unknown'


Create risk groups based on the TCGA classification

Create a risk level column for the TCGA risk level based on the TCGA classification

In [ ]:
df['Risk_level_TCGA'] = np.nan
df['Risk_level_TCGA'] = df['Risk_level_TCGA'].astype(object)

for i in range(len(df)):
	if df['MolClass'].values[i] == 'POLE':
		df['Risk_level_TCGA'][i] = 'Low'
	elif df['MolClass'].values[i] == 'MSI' or df['MolClass'].values[i] == 'NSMP':
		df['Risk_level_TCGA'][i] = 'Medium'
	elif df['MolClass'].values[i] == 'p53':
		df['Risk_level_TCGA'][i] = 'High'
	else:
		df['Risk_level_TCGA'][i] = 'Unknown'


Create a risk level column for the TCGA risk level based on the TCGA classification, but with Biomarkers as a further definition

In [ ]:
df['Risk_level_TCGA_BM'] = np.nan
df['Risk_level_TCGA_BM'] = df['Risk_level_TCGA_BM'].astype(object)

for i in range(len(df)):
	if df['MolClass'].values[i] == 'POLE':
		df['Risk_level_TCGA_BM'][i] = 'Low'
	elif df['MolClass'].values[i] == 'p53':
		df['Risk_level_TCGA_BM'][i] = 'High'
	elif df['MolClass'].values[i] == 'NSMP' or df['MolClass'].values[i] == 'MSI':
		if df['PR_PREOP'].values[i] == 0:
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Mutated':
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['Pre-operative CA-125 level '].values[i] == '>35':
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Wildtype' or df['PR_PREOP'].values[i] > 0 or \
				df['Pre-operative CA-125 level '].values[i] == '<=35':
			df['Risk_level_TCGA_BM'][i] = 'Low'
		else:
			df['Risk_level_TCGA_BM'][i] = 'Unknown'
	else:
		if df['PR_PREOP'].values[i] == 0:
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Mutated':
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['Pre-operative CA-125 level '].values[i] == '>35':
			df['Risk_level_TCGA_BM'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Wildtype' or df['PR_PREOP'].values[i] > 0 or \
				df['Pre-operative CA-125 level '].values[i] == '<=35':
			df['Risk_level_TCGA_BM'][i] = 'Low'
		else:
			df['Risk_level_TCGA_BM'][i] = 'Unknown'


Create a risk level column for the TCGA risk level based on the TCGA classification, but with Biomarkers used only in unkown situations

In [ ]:
df['Risk_level_TCGA_BMNaN'] = np.nan
df['Risk_level_TCGA_BMNaN'] = df['Risk_level_TCGA_BMNaN'].astype(object)

for i in range(len(df)):
	if df['MolClass'].values[i] == 'POLE':
		df['Risk_level_TCGA_BMNaN'][i] = 'Low'
	elif df['MolClass'].values[i] == 'MSI' or df['MolClass'].values[i] == 'NSMP':
		df['Risk_level_TCGA_BMNaN'][i] = 'Medium'
	elif df['MolClass'].values[i] == 'p53':
		df['Risk_level_TCGA_BMNaN'][i] = 'High'
	else:
		if df['PR_PREOP'].values[i] == 0:
			df['Risk_level_TCGA_BMNaN'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Mutated':
			df['Risk_level_TCGA_BMNaN'][i] = 'High'
		elif df['Pre-operative CA-125 level '].values[i] == '>35':
			df['Risk_level_TCGA_BMNaN'][i] = 'High'
		elif df['p53_PREOP'].values[i] == 'Wildtype' or df['PR_PREOP'].values[i] > 0 or \
				df['Pre-operative CA-125 level '].values[i] == '<=35':
			df['Risk_level_TCGA_BMNaN'][i] = 'Low'
		else:
			df['Risk_level_TCGA_BMNaN'][i] = 'Unknown'


Combine the ER PR L1CAM and P53 into one pre and postop column per marker for complete marker columns, to match the paper values

In [ ]:
# Combine ER PR L1CAM and P53 into one pre and postop column per marker
# First check preop, if not there take postop

df['ER_compl'] = np.nan
df['PR_compl'] = np.nan
df['L1CAM_compl'] = np.nan
df['p53_compl'] = np.nan

for i in range(len(df)):
	if not pd.isna(df['ER_PREOP'].values[i]):
		df['ER_compl'][i] = df['ER_PREOP'].values[i]
	elif not pd.isna(df['ER_POSTOP'].values[i]):
		df['ER_compl'][i] = df['ER_POSTOP'].values[i]
	
	if not pd.isna(df['PR_PREOP'].values[i]):
		df['PR_compl'][i] = df['PR_PREOP'].values[i]
	elif not pd.isna(df['PR_POSTOP'].values[i]):
		df['PR_compl'][i] = df['PR_POSTOP'].values[i]
	
	if not pd.isna(df['L1CAM_PREOP'].values[i]):
		df['L1CAM_compl'][i] = df['L1CAM_PREOP'].values[i]
	elif not pd.isna(df['L1CAM_POSTOP'].values[i]):
		df['L1CAM_compl'][i] = df['L1CAM_POSTOP'].values[i]
	
	if not pd.isna(df['p53_PREOP'].values[i]):
		df['p53_compl'][i] = df['p53_PREOP'].values[i]
	elif not pd.isna(df['p53_POSTOP'].values[i]):
		df['p53_compl'][i] = df['p53_POSTOP'].values[i]
	


In [ ]:
df.to_csv('../0.1. Cleaned_data/Tubingen_risk_groups.csv', index=False)

# Creating validation dataset with IHC data
Use IHC data to further define the MSI and MolClass columns, since the categories hide some information

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../0.1. Cleaned_data/Tubingen_risk_groups.csv')
df2 = pd.read_excel('../0. Data/Tubingen/Data_ENDORISK_Validation_MolCLass_TUE_IHC.xlsx')

Check how many df2 patients are double

In [ ]:
# find the double patient id entries in df2 and make them in a list
patient_ids = df2['patient_id'].value_counts(dropna=False)[df2['patient_id'].value_counts(dropna=False) > 1].index.tolist()
len(patient_ids)

From df2 drop the patients who have a double entry in patient_id, drop the one with the least information (more 'Unk')

In [ ]:
# From DF2 drop the patients who have a double entry in patient_id, drop the one with the least information (more 'Unk')
cols = ['mlh1_ihc', 'msh2_ihc', 'msh6_ihc', 'pms2_ihc']

for id in patient_ids:
    rows = df2[df2['patient_id'] == id]
    
    i = 0 # surgery row
    j = 0 # biopsy row
    
    row_surg = rows[rows['tissue_src'] == 'surgery']
    row_biop = rows[rows['tissue_src'] == 'biopsy']

    for i in range(len(cols)):
        column = cols[i]
        if row_surg[column].values[0] == 'Unk':
            i += 1
            j += 0
        if row_biop[column].values[0] == 'Unk':
            i += 0
            j += 1
    
    if i > j:
        df2 = df2.drop(row_surg.index)
    elif j > i:
        df2 = df2.drop(row_biop.index)
    else:
        df2 = df2.drop(row_biop.index)
            

Sort both df and df2 by ascending patient_id and reset the index for comparison

In [ ]:

# Sort both df and df2 by ascending patient_id
df = df.sort_values(by=['Patient ID (L1CAM ID)'])
df2 = df2.sort_values(by=['patient_id'])

# Reset the index of both df and df2
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)


Check if the patient_id's are the same in both df and df2

In [ ]:
# Check if the patient_id's are the same in both df and df2
for i in range(len(df)):
    if df.iloc[i]['Patient ID (L1CAM ID)'] != df2.iloc[i]['patient_id']:
        print('Error: patient_id not the same')
        break

Going through the dataframes, check if the MSI columns are the same in both dataframes, otherwise correct them

In [ ]:
# Create a new column in df2 with the four ihc classes combined into one MMR MSI column
df['MSI'] = np.nan

for i in range(len(df2)):
    row = df2.iloc[i]
    if df2.iloc[i]['patient_id'] != df.iloc[i]['Patient ID (L1CAM ID)']:
        print('Error: patient_id not the same')
        break

    valuemlh1 = row['mlh1_ihc']
    valuemsh2= row['msh2_ihc']
    valuemsh6 = row['msh6_ihc']
    valuepms2 = row['pms2_ihc']
    
    if valuemlh1 == "loss":
        df['MSI'][i] = 'instable'
    elif valuemsh2 == "loss":
        df['MSI'][i] = 'instable'
    elif valuemsh6 == "loss":
        df['MSI'][i] = 'instable'
    elif valuepms2 == "loss":
        df['MSI'][i] = 'instable'
    elif valuemlh1 == 'Unk' and valuemsh2 == 'Unk' and valuemsh6 == 'Unk' and valuepms2 == 'Unk':
        df['MSI'][i] = 'unknown'
    elif valuemlh1 == 'intact' or valuemsh2 == 'intact' or valuemsh6 == 'intact' or valuepms2 == 'intact':
        df['MSI'][i] = 'stable'
    else:
        print('Error: MSI not found at ', i)
        break


Now this data is known the individual TCGA classes can be combined

In [ ]:
# Divide MolClass column into the categories: 'MSI', 'P53', 'POLE', 'NSMP', 'unknown'
df['POLE'] = 'no'
df['P53'] = 'no'
df['NSMP'] = 'no'

x = 0
for i in range(len(df)):
    value = df.iloc[i]['MolClass']
    
    if value == "" or value == " " or value == "unknown":
        df['POLE'] = np.nan
        df['P53'] = np.nan
        df['NSMP'] = np.nan
    elif value == 'POLE':
        df['POLE'][i] = 'yes'
    elif value == 'p53abn':
        df['P53'][i] = 'yes'
    elif value == 'NSMP':
        df['NSMP'][i] = 'NSMP'
    elif value == 'MMRd':
        if df.iloc[i]['MSI'] == 'instable':
            continue
        elif df.iloc[i]['MSI'] == 'stable':
            print('Error: MMRd but not MSI instable, at index: ', i)
            print('Keeping MMRd')
            df['MSI'][i] = 'instable'
	


Check the one that throws an error, indeed there was a mistake here we heard later

In [ ]:
df.iloc[231]

Create a five year survival column

In [ ]:
df['date_of_dx'] = pd.to_datetime(df['date_of_dx'])
df['Date last Follow-Up'] = pd.to_datetime(df['Date last Follow-Up'])

df['five_year_survival'] = 'yes'
        
for i in range(len(df)):
    if df['DSS'].values[i] == 'Endometrial carcinoma':
        df['five_year_survival'][i] = 'no'
    elif df['DSS'].values[i] == 'Other':
        df['five_year_survival'][i] = 'yes'
    else:
        if df['Vitalstatus last Follow-Up'].values[i] == 'Dead' and df['Date last Follow-Up'][i].year - df['date_of_dx'][i].year < 5:
            df['five_year_survival'][i] = 'yes'
        elif df['Vitalstatus last Follow-Up'].values[i] == 'Alive' and df['Date last Follow-Up'][i].year - df['date_of_dx'][i].year > 5:
            df['five_year_survival'][i] = 'yes'
        else:
            df['five_year_survival'][i] = np.nan


Correct the column names to match the model

In [ ]:
df.rename(columns={
	'Grade_PREOP': 'PreoperativeGrade',
	'Grade_POSTOP': 'PostoperativeGrade',
    'MSI': 'MSI',
	'POLE': 'POLE',
	'P53':'TP53', # Correctly name the P53 column to the gene name
    'p53_compl': 'p53', # IHC p53, which should be used (for now)
    'ER_compl':'ER',
    'PR_compl':'PR',
    'L1CAM_compl':'L1CAM',
    'Pre-operative platelet count ':'Platelets',
	'Myometrial invasion':'MyometrialInvasion',
	'LNM_obs':'LNM',
	'CA125_bi':'CA125',
	"Adjuvant therapy":'Therapy',
	"Reucrrence":'Recurrence',
	'five_year_survival':'Survival5yr',
}, inplace=True)

Replace the labels in the columns with the correct labels

In [ ]:
#  Replace the values in the columns with the correct labels
df.replace({'Grade 1': 'grade 1', 'Grade 2': 'grade 2', 'Grade 3': 'grade 3'}, inplace=True)
df['MSI'].replace({"instable":"yes", "stable":"no"}, inplace=True)
df['POLE'].replace({"yes":"yes", "no":"no"}, inplace=True)
df['p53'].replace({"Mutated":"mutant", "Wildtype":"wildtype"}, inplace=True)
df['CA125'].replace({"<=35":"lt_35", ">35":"ge_35"}, inplace=True)
df['ER'].replace({1:"positive", 0:"negative"}, inplace=True)
df['PR'].replace({1:"positive", 0:"negative"}, inplace=True)
df['L1CAM'].replace({1:"positive", 0:"negative"}, inplace=True)
df['Platelets'].replace({"<400":"lt_400", ">=400":"ge_400"}, inplace=True)
df['MyometrialInvasion'].replace({"No invasion":"lt_50","<50%":"lt_50", ">50%":"ge_50"}, inplace=True)
df['Therapy'].replace({"RTx":"radiotherapy", "CTx":"chemotherapy", "both":"chemoradiotherapy", "Both":"chemoradiotherapy", np.nan:"none"}, inplace=True)
df['LNM'].replace({"negative":"no", "positive":"yes"}, inplace=True)
df['Histology'] = df['Histotype'].apply(lambda x: 'endometrioid' if x == 'Endometrioid' else (np.nan if pd.isna(x) else 'non-endometrioid'))
df

Create a column for the chemotherapy and radiotherapy from the therapy column

In [ ]:
# Create Chemo and radiotherapy columns from Therapy
df["Chemotherapy"] = np.nan
df["Radiotherapy"] = np.nan

for i in range(len(df)):
	if df['Therapy'].values[i] == 'chemotherapy':
		df['Chemotherapy'][i] = 'yes'
		df['Radiotherapy'][i] = 'no'
	elif df['Therapy'].values[i] == 'radiotherapy':
		df['Chemotherapy'][i] = 'no'
		df['Radiotherapy'][i] = 'yes'
	elif df['Therapy'].values[i] == 'chemoradiotherapy':
		df['Chemotherapy'][i] = 'yes'
		df['Radiotherapy'][i] = 'yes'
	elif df['Therapy'].values[i] == 'none':
		df['Chemotherapy'][i] = 'no'
		df['Radiotherapy'][i] = 'no'
	else:
		df['Chemotherapy'][i] = np.nan
		df['Radiotherapy'][i] = np.nan

save the dataframe

In [ ]:
df.to_csv('../0.1. Cleaned_data/Tubingen_Validation_wMSI.csv', index=False)
